<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->

# Interactive Beam Running on Flink

In [1]:
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.runners.portability import portable_runner
from apache_beam.options import pipeline_options

In [2]:
options = pipeline_options.PipelineOptions()
options.view_as(pipeline_options.PortableOptions).job_endpoint = 'localhost:8099'
options.view_as(pipeline_options.SetupOptions).sdk_location = 'container'
options.view_as(pipeline_options.DebugOptions).experiments = 'beam_fn_api'

In [3]:
cache_dir = "gs://bucket-name/path/to/dir"
underlying_runner = portable_runner.PortableRunner()
runner = interactive_runner.InteractiveRunner(underlying_runner=underlying_runner, cache_dir=cache_dir)

In [4]:
p = beam.Pipeline(runner=runner, options=options)
init_pcoll = p |  beam.Create(range(10))
squares = init_pcoll | 'Square' >> beam.Map(lambda x: x*x)
cubes = init_pcoll | 'Cube' >> beam.Map(lambda x: x**3)
result = p.run()
result.wait_until_finish()

Running...

Using 0 cached PCollections
Executing 8 of 3 transforms.

Cube produced {27, 512, 216, 125, 729, ...}

Square produced {16, 25, 64, 36, 81, ...}

Create produced {1, 3, 5, 0, 6, ...}

In [5]:
class AverageFn(beam.CombineFn):
  def create_accumulator(self):
    return (0.0, 0)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):
    sums, counts = zip(*accumulators)
    return sum(sums), sum(counts)

  def extract_output(self, sum_count):
    (sum, count) = sum_count
    return sum / count if count else float('NaN')

In [6]:
average_square = squares | 'Average Square' >> beam.CombineGlobally(AverageFn())
average_cube = cubes | 'Average Cube' >> beam.CombineGlobally(AverageFn())
result = p.run()

Running...

Using 2 cached PCollections
Executing 8 of 5 transforms.

Average Cube produced {202.5}

Cube produced {27, 512, 216, 125, 729, ...}

Square produced {16, 25, 64, 36, 81, ...}

Average Square produced {28.5}